In [1]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data2 = np.genfromtxt('data2.dat', delimiter='  ')

y = data2[:, 0]
x = data2[:, 1]

In [3]:
# MLE
def loglike(params, *args):
    y, x = args
    theta0, theta1, sigma = params
    n = len(y)
    e = np.log(y) - theta0 - x**theta1
    loglike = -n/2*np.log(2*np.pi*sigma**2) - 1/(2*sigma**2)*sum(e**2)
    return -loglike

In [4]:
from scipy.optimize import minimize
res = minimize(loglike, [0,0,1], args=(y,x), method='L-BFGS-B', bounds=((None, None), (None, None), (1e-10, None)))

In [6]:
print(res.x)

[3.06593632 1.02719735 0.96334309]


In [7]:
print(res.hess_inv.todense())

[[ 0.07441462 -0.07400939 -0.01694223]
 [-0.07400939  0.08654626  0.01794028]
 [-0.01694223  0.01794028  0.01975501]]


In [8]:
vcv_mle = res.hess_inv.todense()

In [9]:
stderr_theta0 = np.sqrt(vcv_mle[0,0])
stderr_theta1 = np.sqrt(vcv_mle[1,1])
stderr_sigma = np.sqrt(vcv_mle[2,2])
print('Standard error for theta0 is ', stderr_theta0)
print('Standard error for theta1 is ', stderr_theta1)
print('Standard error for sigma is ', stderr_sigma)

Standard error for theta0 is  0.27279042984378365
Standard error for theta1 is  0.2941874498915475
Standard error for sigma is  0.14055253238526072


In [10]:
def bootstrap(n = 100):
    theta0 = []
    theta1 = []
    sigma = []
    for _ in range(n):
        sample = np.random.choice(range(len(y)), len(y), replace=True)
        res = minimize(loglike, [0,0,1], args=(y[sample],x[sample]), method='L-BFGS-B', bounds=((None, None), (None, None), (1e-10, None)))
        theta0.append(res.x[0])
        theta1.append(res.x[1])
        sigma.append(res.x[2])
    return theta0, theta1, sigma

In [11]:
theta0_list, theta1_list, sigma_list = bootstrap(n = 100)

In [14]:
np.array(theta0_list).std()

0.1762667213817594

In [12]:
np.array(theta1_list).std()

0.21335101634782958

In [13]:
np.array(sigma_list).std()

0.058287179360058845